# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [56]:
# Libraries

import pandas as pd
import numpy as np


# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [57]:
# your code here

admissions_db = pd.read_csv("Admission_Predict.csv")
admissions_db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Remove trailing spaces at the end of the column names if there are any.


In [58]:
# your code here

#end of the column:

#admissions_db.columns # there are spaces in the 'LOR ' and 'Chance of Admit '
admissions_db.columns = admissions_db.columns.str.strip()
admissions_db.columns


Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [59]:
# your code here

admissions_db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [60]:
# your code here

admissions_db.isnull()

admissions_db.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [61]:
# your code here


#admissions_index = admissions_db.set_index('Serial No.')

#admissions_db.set_index('Serial No.',drop=False) 
# spend a lot of time trying to do in another way, but i don´t know how????
#error "None of ['Serial No.'] are in the columns - but i already removed trailing spaces.



admissions_db.set_index(['Serial No.'],inplace=True)
admissions_db.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [62]:
# your code here

print(admissions_db["GRE Score"].unique().sum())

print(admissions_db["CGPA"].unique().sum())

print(len(admissions_db))

#i didn´t understand the question... They can be my "id" for the data, right? but how can i show this?

15482
1449.99
385


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [63]:
# your code here


#admissions_db.reset_index("GRE + CGPA", inplace = True) 


admissions_db.set_index(['GRE Score','CGPA'],inplace=True)
admissions_db.head()



,,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,
337,9.65,118,4,4.5,4.5,1,0.92
316,8.00,104,3,3.0,3.5,1,0.72
322,8.67,110,3,3.5,2.5,1,0.80
314,8.21,103,2,2.0,3.0,0,0.65
330,9.34,115,5,4.5,3.0,1,0.90


In [64]:
admissions_db.reset_index(inplace=True)
admissions_db.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
1,316,8.00,104,3,3.0,3.5,1,0.72
2,322,8.67,110,3,3.5,2.5,1,0.80
3,314,8.21,103,2,2.0,3.0,0,0.65
4,330,9.34,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [65]:
# your code here


#admissions_db["CGPA"] > 9

#admissions_db["Research"] >= 1

#and = &

admissions_db[(admissions_db["CGPA"] > 9) & (admissions_db["Research"] >= 1)]

#101 rows

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
4,330,9.34,115,5,4.5,3.0,1,0.90
10,328,9.10,112,4,4.0,4.5,1,0.78
19,328,9.50,116,5,5.0,5.0,1,0.94
20,334,9.70,119,5,5.0,4.5,1,0.95
...,...,...,...,...,...,...,...,...
379,329,9.23,111,4,4.5,4.0,1,0.89
380,324,9.04,110,3,3.5,3.5,1,0.82
381,325,9.11,107,3,3.0,3.5,1,0.84
382,330,9.45,116,4,5.0,4.5,1,0.91


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [66]:
# your code here


admissions_db[(admissions_db["CGPA"] > 9) & (admissions_db["SOP"] < 3.5)]

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
28,338,9.40,118,4,3.0,4.5,1,0.91
62,327,9.02,114,3,3.0,3.0,0,0.61
140,326,9.11,114,3,3.0,3.0,1,0.83
217,324,9.01,111,4,3.0,3.0,1,0.82
381,325,9.11,107,3,3.0,3.5,1,0.84


In [67]:

admissions_db["Chance of Admit"].mean()

0.7239220779220779

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [68]:
def standardize(col):
    """
    This function takes a column from a dataframe #any column
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """

#needed help to get here:
    

    # your code here
    col_std = []
    for c in col :
        col_std.append((c - np.mean(col)) / np.std(col))      
    return col_std


#simple way:

    #return (col - col.mean())/col.std()
    
#deciding factor column - each one is that?
#write a standardization function: takes any column and return. If the standardized value is above 0.8, the student will be accepted.


Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [69]:
# your code here

#create 3 new columns

admissions_db["CGPA_std"] = standardize(admissions_db["CGPA"])
admissions_db["GRE_std"] = standardize(admissions_db["GRE Score"])
admissions_db["LOR_std"] = standardize(admissions_db["LOR"])

admissions_db.head()




,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338


We will generate the decision choice at random using the code below. Please run the cell.

In [70]:
# Libraries
from random import choices

In [71]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std'] #only the tree new columns:

decision_choice = choices(std_columns, k=admissions_db.shape[0]) #my k is the data

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [72]:
# your code here
#got confused... 

admissions_db["deciding_column"] = admissions_db.lookup(admissions_db.index, decision_choice)
admissions_db.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,0.076840
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [73]:
# your code here

#admissions_db['decision'] = admissions_db['deciding_column'] (missing something in here) x if x > 0.8, 1, 0
#admissions_db.head()

#Ana´s way:                                          
admissions_db["decision"] = admissions_db["deciding_column"].apply(lambda x: 1 if x >0.8 else 0)
admissions_db.head(10)

#1 is only for those who have more than 0.8

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297,0
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0
5,321,8.20,109,3,3.0,4.0,1,0.75,-0.660055,0.369672,0.635019,-0.660055,0
6,308,7.90,101,2,3.0,4.0,0,0.68,-1.158723,-0.756445,0.635019,0.635019,0
7,302,8.00,102,1,2.0,1.5,0,0.50,-0.992501,-1.276191,-2.155874,-2.155874,0
8,323,8.60,108,3,3.5,3.0,0,0.45,0.004836,0.542921,-0.481338,0.542921,0
9,327,9.00,111,4,4.0,4.5,1,0.84,0.669726,0.889419,1.193197,1.193197,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [74]:
# your code here


admissions_db["decision"].value_counts()

#We will have 88 admissions.

0    297
1     88
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [77]:
# your code here

#admissions_db.columns

admissions_db.columns = admissions_db.columns.str.strip('.').str.replace(' ','_').str.lower()
#admissions_db.head()

admissions_db.columns


Index(['gre_score', 'cgpa', 'toefl_score', 'university_rating', 'sop', 'lor',
       'research', 'chance_of_admit', 'cgpa_std', 'gre_std', 'lor_std',
       'deciding_column', 'decision'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [78]:
# your code here

#give all student with a university rating of 4 or higher a 10 point boost [GRE]
#/4 using cut

#ana´s helped:
# 1 - create a new column and put the 10 points 
# 2- divided using bins and putting labels

admissions_db["adjusted_gre"] = admissions_db["gre_score"][admissions_db.university_rating >= 4] + 10

admissions_db["adjusted_gre"] = pd.cut(x=admissions_db["gre_score"], bins = 4, labels = ["Low", "Average", "Good", "Excellent"])

admissions_db.head(10)


,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1,Excellent
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,0.076840,0,Good
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297,0,Good
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0,Average
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0,Excellent
5,321,8.20,109,3,3.0,4.0,1,0.75,-0.660055,0.369672,0.635019,-0.660055,0,Good
6,308,7.90,101,2,3.0,4.0,0,0.68,-1.158723,-0.756445,0.635019,0.635019,0,Average
7,302,8.00,102,1,2.0,1.5,0,0.50,-0.992501,-1.276191,-2.155874,-2.155874,0,Low
8,323,8.60,108,3,3.5,3.0,0,0.45,0.004836,0.542921,-0.481338,0.542921,0,Good
9,327,9.00,111,4,4.0,4.5,1,0.84,0.669726,0.889419,1.193197,1.193197,1,Good
